This notebook generates dataset according to CrossDock 100k subset
- The goal is to use full protein from original CrossDock dataset instead of the version with 12A cutoff.
- pockets not from PDBBind set are removed 

You need to have CrossDock subset and full set downloaded to run this script.

In [12]:
import os
from glob import glob
import torch
from tqdm import tqdm
from collections import defaultdict 
import sys

sys.path.append('../')

from src.tacogfn.eval import docking
from src.tacogfn.utils import molecules

There are 24478 receptors

In [13]:
pocket_paths = glob('../dataset/crossdock/*_rec.pdb')
len(pocket_paths)

44646

There are 21231 ligands

In [14]:
ligand_paths = glob('../dataset/crossdock/*_lig.pdb')
len(ligand_paths)

22288

In [27]:
pocket_ids = [i.split('/')[-1].split('_rec')[0] for i in pocket_paths]
lig_ids = [i.split('/')[-1].split('_')[0] for i in ligand_paths]

Convert all pdb ligands into sdf

In [31]:
# for lig_path in tqdm(ligand_paths):
#     try:
#         molecules.convert_pdb_to_sdf(lig_path, lig_path.replace('.pdb', '.sdf'))
#     except:
#         print(lig_path)

Prepare for docking

In [18]:
# for ligand_sdf_path in tqdm(ligand_sdf_paths):
#     if not os.path.exists(ligand_sdf_path.replace('.sdf', '_h.sdf')):
#         molecules.add_implicit_hydrogens_to_sdf(
#             ligand_sdf_path,
#             ligand_sdf_path.replace('.sdf', '_h.sdf')
#         )
        
#     if not os.path.exists(ligand_sdf_path.replace('.sdf', '.pdbqt')):
#         docking._prepare_ligand(
#             ligand_sdf_path.replace('.sdf', '_h.sdf'),
#             ligand_sdf_path.replace('.sdf', '.pdbqt')
#         )

In [19]:
# for pocket_path in tqdm(pocket_paths):
#     target_path = pocket_path.replace('.pdb', '.pdbqt').replace('crossdock', 'crossdock_pdbqt')
#     if not os.path.exists(target_path):
#         docking._prepare_receptor(
#             pocket_path,
#             target_path
#         )

We now match ligands to pockets based on filenames and resolve based on docking scores

In [24]:
# Pocket2Mol pdb ids
split_by_name = torch.load('../dataset/split_by_name.pt')
train_pdb_ids = [val[0].split('/')[-1].split('_rec')[0] for val in split_by_name['train']]
val_pdb_ids = [val[0].split('/')[-1].split('_rec')[0] for val in split_by_name['test']]
pocket2mol_split = split_by_name['train'] + split_by_name['test']
pocket2mol_pdb_ids = set(train_pdb_ids + val_pdb_ids)

In [35]:
cross_dock_pocket_ids = set(pocket_ids)

In [48]:
pocket_to_ligands = defaultdict(list)

for pair in pocket2mol_split:
    pdb_id = pair[0].split('/')[-1].split('_rec')[0]
    ligand_path = pair[1]

    pocket_to_ligands[pdb_id].append(ligand_path)

In [50]:
pocket_to_ligand = {}
for key, value in pocket_to_ligands.items():
    pocket_to_ligand[key] = value[0]

In [42]:
for pdb_id in pocket2mol_pdb_ids:
    print(pdb_id)
    
    matched_pocket_paths = [p for p in pocket_paths if pdb_id in p]
    matched_ligand_paths = [l for l in ligand_paths if pdb_id in l]
    
    if pdb_id not in cross_dock_pocket_ids:
        print(pdb_id)
        
    print(matched_pocket_paths, matched_ligand_paths)
    break

1uf4_A
['../dataset/crossdock/1uf4_A_rec.pdb'] []


In [21]:
def find_best_matches(pocket_paths, ligand_paths):
    """Given a list of pocket and ligand paths, find the best matching ligand for pocket"""
    result = {}
    for pocket_path in pocket_paths:
        min_score = - 0.5 # score must be less than -1.0 (default)
        for ligand_path in ligand_paths:
            sdf_path = ligand_path.replace('pdb', 'sdf')
            if os.path.exists(sdf_path):
                score = docking.compute_docking_score_from_sdf(
                    pocket_path,
                    sdf_path,
                    keep_temp_folder=False,
                    score_only=True
                )
                if score is not None and score < min_score:
                    min_score = score
                    result[pocket_path] = ligand_path
    return result
            

SyntaxError: invalid syntax (2142193334.py, line 11)

In [22]:
pocket_to_ligands = {}

for pocket_id in tqdm(pocket_ids):
    # find matched pocket and ligand paths
    matched_pocket_paths = [p for p in pocket_paths if pocket_id in p]
    matched_ligand_paths = [l for l in ligand_paths if pocket_id in l]
    
    # if only one match, add to dict
    if len(matched_pocket_paths) == 1 and len(matched_ligand_paths) == 1:
        pocket_to_ligands[matched_pocket_paths[0]] = matched_ligand_paths[0]  
    # if more than one match, find best match         
    else:
        results = find_best_matches(matched_pocket_paths, matched_ligand_paths)
        for pocket_path, ligand_path in results.items():
            pocket_to_ligands[pocket_path] = ligand_path
            
torch.save(pocket_to_ligands, 'crossdock_pocket_to_native_ligand.pt')

  0%|          | 0/44646 [00:00<?, ?it/s]

adding gasteiger charges to peptide
adding gasteiger charges to peptide


  0%|          | 2/44646 [00:01<11:03:03,  1.12it/s]

adding gasteiger charges to peptide
adding gasteiger charges to peptide


  0%|          | 3/44646 [00:03<14:30:56,  1.17s/it]

...

  0%|          | 5/44646 [00:11<32:28:28,  2.62s/it][10:43:25] Explicit valence for atom # 0 N, 4, is greater than permitted
[10:43:25] ERROR: Could not sanitize molecule ending on line 40
[10:43:25] ERROR: Explicit valence for atom # 0 N, 4, is greater than permitted
Traceback (most recent call last):
  File "/home/tsa87/anaconda3/envs/tacogfn/bin/mk_prepare_ligand.py", line 286, in <module>
    mol_supplier = parsers[ext](input_molecule_filename, removeHs=False) # input must have explicit H
OSError: File error: Invalid input file /tmp/tmpoiusw7rb/ligand.sdf


WARNING! 5fw0_rec has 9  atoms with alternate locations named ['A']!

WARNING! using alternate location A
Sorry, there are no Gasteiger parameters available for atom 5fw0_rec:A:ZN 900:ZN


  0%|          | 6/44646 [00:13<26:53:24,  2.17s/it]

WARNING! 5fw0_rec has 9  atoms with alternate locations named ['A']!

WARNING! using alternate location A
Sorry, there are no Gasteiger parameters available for atom 5fw0_rec:A:ZN 900:ZN


KeyboardInterrupt: 

In [10]:
pocket_to_ligands = torch.load('../dataset/crossdock_pocket_to_native_ligand.pt')

Try to merge with Pocket2Mol dataset - bring more dataset

In [40]:
# # These are the pdb ids that are in Pocket2Mol but not in CrossDock
# # We will download these pdb ids from PDB
# pdb_ids_to_download = []
# for pdb_id in pocket2mol_pdb_ids:
#     if not os.path.exists(os.path.join('../dataset/crossdock', pdb_id + '_rec.pdb')):
#         pdb_ids_to_download.append(pdb_id.split('_')[0])
# with open('../misc/pdb_ids_in_pocket2mol_to_download.txt', 'w') as file:
#     file.write(','.join(pdb_ids_to_download))

In [41]:
# CrossDock pdb ids
pocket_to_ligands = torch.load('../dataset/crossdock_pocket_to_native_ligand.pt')
existing_pdb_ids = set([v.split('/')[-1].split('_rec')[0] for v in list(pocket_to_ligands.keys())])

In [44]:
not_found_pdb_ids = set([i for i in pocket2mol_pdb_ids if i not in existing_pdb_ids])

In [45]:
not_found_pdb_ids 

{'14gs_A',
 '1a2g_A',
 '1ai4_A',
 '1h0i_A',
 '1k9t_A',
 '1l3l_A',
 '1umd_B',
 '2cy0_A',
 '2e24_A',
 '2gns_A',
 '2pc8_A',
 '2pqw_A',
 '2rma_A',
 '2zen_A',
 '3ej8_A',
 '3jyh_A',
 '3li4_A',
 '3pdh_A',
 '3pnm_A',
 '4g3d_B',
 '4gvd_A',
 '4keu_A',
 '4qlk_A',
 '5b08_A',
 '5q0k_A',
 '5tjn_A',
 '5w2g_A'}

This section just renames PDB files

In [15]:
# for pdb_id in pdb_ids_to_download:
#     assert os.path.exists(f'../dataset/non_pdbbind/{pdb_id}.pdb')

In [16]:
# Rename the PDB files 
# for pdb_chain_id in tqdm(not_found_val):
#     pdb_id = pdb_chain_id.split('_')[0]
#     os.system(f'cp ../dataset/non_pdbbind/{pdb_id}.pdb ../dataset/non_pdbbind/{pdb_chain_id}.pdb')

In [17]:
# for pdb_id in pdb_ids_to_download:
#     os.system(f'rm ../dataset/non_pdbbind/{pdb_id}.pdb')

In [18]:
not_found_val = list(not_found_val)

In [19]:
not_found_in_both = []
for pdb_id in not_found_val:
    try:
        assert os.path.exists(f'../dataset/non_pdbbind/{pdb_id}.pdb') or os.path.exists(f'../dataset/crossdock/{pdb_id}_rec.pdb')
    except:
        not_found_in_both.append(pdb_id)
len(not_found_in_both)

27

Let's match these difficult to match pockets

In [20]:
pairs_of_interests = [
    pair for pair in pocket2mol_split if pair[0].split('/')[-1].split('_')[0] in set(pdb_ids_to_download)
]

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from src.tacogfn.utils import molecules

In [37]:
# from src.tacogfn.utils import molecules


# for pocket_id in tqdm(not_found_val):
#     if pocket_id in not_found_in_both:
#         continue
    
#     pocket_path1 = os.path.join('../dataset/crossdock/', pocket_id + '_rec.pdb')
#     pocket_path2 = os.path.join('../dataset/non_pdbbind/', pocket_id + '.pdb')
#     if os.path.exists(pocket_path1):
#         pocket_path = pocket_path1
#     else:
#         pocket_path = pocket_path2
        
#     # find matched pocket and ligand paths
#     matched_pocket_paths = [v[0] for v in pocket2mol_split if pocket_id in v[0]]
#     matched_ligand_paths = [v[1] for v in pocket2mol_split if pocket_id in v[1]]

#     aligned_crossdock_pocket_paths = None
#     for crossdock_pocket_path in matched_pocket_paths:
#         try:
#             crossdock_pocket_path = os.path.join('../dataset/pocket2mol_dataset/', crossdock_pocket_path.split('/')[-1])
#             rot_mat, original_center, new_center = molecules.get_rototranslation_alignment(
#                 pocket_path, 
#                 crossdock_pocket_path,
#                 max_rmse=2.0
#             )
#             aligned_crossdock_pocket_paths = crossdock_pocket_path
#         except KeyboardInterrupt:
#             raise KeyboardInterrupt
#         except Exception:
#             print(crossdock_pocket_path)
#             print(pocket_path)
#             print(crossdock_pocket_path.replace('.pdb', '_aligned.pdb'))
        
#             molecules.transform_pdb(
#                 crossdock_pocket_path, 
#                 crossdock_pocket_path.replace('.pdb', '_aligned.pdb'),
#                 rot_mat,
#                 original_center,
#                 new_center
#             )


        
#         #     print(crossdock_pocket_path)
#         #     print(pocket_path)
#         #     continue
    
            

#     # print(aligned_crossdock_pocket_paths)
#     # print(matched_ligand_paths)


In [ ]:

# for pair in tqdm(pairs_of_interests):
#     pdb_id = pair[0].split('/')[-1].split('_')[0]
    
#     # Downloaded from PDBBind
#     pocket_path = os.path.join('../dataset/non_pdbbind/', pdb_id + '.pdb')
    
#     # Original Pocket2Mol dataset
#     original_pocket_path = os.path.join('../dataset/pocket2mol_dataset/', pair[0].split('/')[-1])
#     ligand_path = os.path.join('../dataset/pocket2mol_dataset/', pair[1].split('/')[-1])
    
#     # Processed ligand path
#     ligand_sdf_id = ligand_path.split('/')[-1].split('lig')[0] + 'lig' 
#     processed_ligand_path = os.path.join('../dataset/non_pdbbind/', f'{ligand_sdf_id}.sdf')
    
#     try:
#         rot_mat, original_center, new_center = molecules.get_rototranslation_alignment(
#             pocket_path, 
#             original_pocket_path,
#             max_rmse=2.0
#         )

#         molecules.transform_sdf(
#             input_file=ligand_path, 
#             output_file=processed_ligand_path, 
#             rotation_matrix=rot_mat, 
#             original_center=original_center, 
#             new_center=new_center
#         )

#         score = docking.compute_docking_score_from_sdf(
#             pocket_path,
#             processed_ligand_path,
#             keep_temp_folder=False,
#             score_only=True
#         )
#     except:
#         continue
    
#     if score is not None and score < -2.0:
#         pocket_to_ligands[pocket_path] = processed_ligand_path
            
#     print(f'{pdb_id} {score}')

  0%|          | 0/10361 [00:00<?, ?it/s]

WARNING! 2jau has 20  atoms with alternate locations named ['A', 'B']!

WARNING! using alternate location A
adding gasteiger charges to peptide


  0%|          | 1/10361 [00:00<2:15:05,  1.28it/s]

2jau -5.77317
WARNING! 5pnx has 208  atoms with alternate locations named ['A', 'B', 'C', 'D']!

WARNING! using alternate location A
adding gasteiger charges to peptide


  0%|          | 2/10361 [00:01<2:23:41,  1.20it/s]

...

adding gasteiger charges to peptide


  0%|          | 50/10361 [00:31<1:21:11,  2.12it/s]

1v3x -9.57747
WARNING! 5nak has 96  atoms with alternate locations named ['A', 'B']!

WARNING! using alternate location A
adding gasteiger charges to peptide


  1%|          | 53/10361 [00:33<1:28:15,  1.95it/s]

5nak -6.98287
WARNING! 4fr8 has 392  atoms with alternate locations named ['A', 'B']!

WARNING! using alternate location A
adding gasteiger charges to peptide


In [ ]:
len(set(not_found_val))

2244

In [ ]:
len(set(pdb_ids_to_download))

2149

Archive Code

In [ ]:
# # Convert path to pocket id and ligand id
# get_pocket_id_and_ligand_id = lambda path: (path.split('/')[1].split('_rec')[0], path.split('/')[1].split('rec_')[1].split('_lig')[0])

In [ ]:
# # Convert path into pocket ligand pairs
# pairs = [get_pocket_id_and_ligand_id(path) for path, _ in merged_split_file]

In [ ]:
# map pocket to list of ligands
# pocket_to_ligands = defaultdict(list)

# for pocket, ligand in pairs:
#     pocket_to_ligands[pocket].append(ligand)

In [ ]:
# map pocket to the native ligand, if native ligand is not in the list of ligands, then use the first ligand
# pocket_to_best_ligand = {}
# for pocket, ligand_list in pocket_to_ligands.items():
#     pocket_id = pocket.split('_')[0]
    
#     # default to first ligand
#     pocket_to_best_ligand[pocket] = ligand_list[0]
    
#     for ligand in ligand_list:
#         if ligand.split('_')[0] == pocket_id:
#             pocket_to_best_ligand[pocket] = ligand

In [ ]:
# # Convert the pocket id to pdb path in CrossDock
# # Create a map of pocket id to pdb path
# pocket_id_to_pdb_path = {}

# for pocket_rec_id, _ in pocket_to_best_ligand.items():
#     path = f'../dataset/crossdock/{pocket_rec_id}_rec.pdb'
    
#     if os.path.exists(path):
#         pocket_id_to_pdb_path[pocket_rec_id] = path
#     else:
#         pocket_id_to_pdb_path[pocket_rec_id] = None  

In [ ]:
# num_pockets_not_in_pdbbind = len({k: v for k, v in pocket_id_to_pdb_path.items() if v is None})
# print(f'Number of pockets not in pdbbind: {num_pockets_not_in_pdbbind}')

Number of pockets not in pdbbind: 831


In [ ]:
# # Convert the pocket_to_best_ligand in paths
# protein_path_to_ligand_path = {}

# for pocket_path, ligand_path in merged_split_file:
#     pocket_id, ligand_id = get_pocket_id_and_ligand_id(pocket_path)
    
#     best_ligand_id = pocket_to_best_ligand[pocket_id]
#     protein_path = pocket_id_to_pdb_path[pocket_id]
#     if best_ligand_id == ligand_id and protein_path is not None:
#         protein_path_to_ligand_path[protein_path] = os.path.join('../dataset/crossdocked_pocket10/', ligand_path)    

In [ ]:
# # move the files into a processed folder 
# # make sure we align the pocket and the whole protein - and then we can adjust the ligand accordingly

# processed_protein_id_to_ligand_id = {}
# failed_dict = {}

# for protein_path, ligand_path in tqdm(protein_path_to_ligand_path.items()):
    
#     pocket_path = ligand_path.replace('.sdf', '_pocket10.pdb')
#     assert os.path.exists(pocket_path)
    
#     protein_rec_id = os.path.splitext(os.path.basename(protein_path))[0]
#     protein_rec_filename = f'{protein_rec_id}.pdb'
    
#     ligand_sdf_id = ligand_path.split('/')[-1].split('lig')[0] + 'lig' 
    
#     processed_protein_path = os.path.join('../dataset/processed/proteins/', protein_rec_filename)
#     processed_ligand_path = os.path.join('../dataset/processed/ligands/', f'{ligand_sdf_id}.sdf')
    
    
#     os.system(f'cp {protein_path} {processed_protein_path}')
#     # try:
#     try:
#         rot_mat, original_center, new_center = molecules.get_rototranslation_alignment(
#             protein_path, 
#             pocket_path,
#         )

#         molecules.transform_sdf(
#             input_file=ligand_path, 
#             output_file=processed_ligand_path, 
#             rotation_matrix=rot_mat, 
#             original_center=original_center, 
#             new_center=new_center
#         )
#     except:
#         os.system(f'cp {ligand_path} {processed_ligand_path}')
#         print(processed_protein_path)
#         print(processed_ligand_path)
#         break
    
#     processed_protein_id_to_ligand_id[protein_rec_id] = ligand_sdf_id
#     # except:
#     #     failed_dict[protein_rec_id] = ligand_sdf_id

  0%|          | 2/14476 [00:00<11:39, 20.69it/s]

../dataset/processed/proteins/5acc_A_rec.pdb
../dataset/processed/ligands/5acc_A_rec_5acc_ke9_lig.sdf


In [ ]:
# # Get the train and test split for CrossDock pockets
# train_pockets = set([get_pocket_id_and_ligand_id(path)[0] for path, _ in split_file['train']])
# test_pockets = set([get_pocket_id_and_ligand_id(path)[0] for path, _ in split_file['test']])

In [ ]:
# # Create a new split file for CrossDock 

# new_split_file = {'train': {}, 'test': {}}

# for pocket_id, ligand_id in processed_protein_id_to_ligand_id.items():
#     pocket_id = pocket_id.split('_rec')[0]
#     if pocket_id in train_pockets:
#         new_split_file['train'][pocket_id] = ligand_id
#     elif pocket_id in test_pockets:
#         new_split_file['test'][pocket_id] = ligand_id
#     else:
#         raise Exception('Pocket not in train or test split')


In [ ]:
# torch.save(new_split_file, '../dataset/processed/processed_split_file.pt')